# An Example of Nowcasting with DLT

This session demos a nowcasting problem using Orbit to explains the data. Since this is a nowcasting problem, we can adopt concurrent information to explain the data such as the search queries and stock market indices happened in the same period. Such idea was also explored in the Bayesian Structural Time-Series(BSTS) paper [Scott and Varan (2013)](https://people.ischool.berkeley.edu/~hal/Papers/2013/pred-present-with-bsts.pdf). In particular, the session covers 

- orbit installation
- a forecasting task on iclaims dataset
- a simple DLT model
- DLT model with regression settings
- models diagnostic
- regression with informative priors

For more examples, check out the [Github](https://github.com/uber/orbit).

# Installation

Read the requirements file and make sure you have a C++ compiler for PyStan

In [ ]:
!pip install git+https://github.com/uber/orbit.git@dev
!pip install matplotlib==3.1.3
!pip install scikit-learn

In [ ]:
%matplotlib inline

import os
import platform
import numpy as np
import matplotlib.pyplot as plt

import orbit
from orbit.utils.dataset import load_iclaims
from orbit.models import DLT
from orbit.diagnostics.plot import plot_predicted_data, plot_predicted_components
from orbit.diagnostics.metrics import smape
from sklearn.preprocessing import MinMaxScaler

In [ ]:
seed=2022

In [ ]:
print(orbit.__version__)
print(os.name)
print(platform.system())

In [ ]:
def mae(x, y):
    return np.mean(np.abs(x - y))

# US Weekly Initial Claims

The *iclaims* data contains the weekly initial claims for US unemployment benefits against a few related google trend queries (unemploy, filling and job) 
from Jan 2010 - June 2018. This dataset was used in the original Bayesian Structural Time-Series paper [Scott and Varan (2013)](https://people.ischool.berkeley.edu/~hal/Papers/2013/pred-present-with-bsts.pdf) as well.

Number of claims are obtained from [Federal Reserve Bank of St. Louis](https://fred.stlouisfed.org/series/ICNSA) while regressors such as google queries are obtained through [Google Trends API](https://trends.google.com/trends/?geo=US).

In order to use this data to nowcast the US unemployment claims considering the impact of COVID-19, we extended the dataset to Jan 2021 and added the [S&P 500 (^GSPC)](https://finance.yahoo.com/quote/%5EGSPC/history?period1=1264032000&period2=1611187200&interval=1wk&filter=history&frequency=1wk&includeAdjustedClose=true) and [VIX](https://finance.yahoo.com/quote/%5EVIX/history?p=%5EVIX) Index historical data for the same period.

In [ ]:
# load data
df = load_iclaims(end_date='2021-01-03')
df = df[['week', 'claims', 'trend.unemploy', 'trend.job', 'sp500', 'vix']]

For SP500 index, it is more reasonable to examine the change of the index which naturally captures the shock of market's belief of the economics in future periods.

In [ ]:
df[['sp500']] = df[['sp500']].diff()
df = df[1:].reset_index(drop=True)

date_col = 'week'
response_col = 'claims'
df.dtypes

In [ ]:
df.head(5)

We can see from the charts below, there are seasonlity, trend, and as well as a huge changpoint due the impact of COVID-19.

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(16, 8))
axs[0, 0].plot(df['week'], df['claims'])
axs[0, 0].set_title('Unemployment Claims')
axs[0, 1].plot(df['week'], df['trend.unemploy'], 'tab:orange')
axs[0, 1].set_title('Google trend - unemploy')
axs[1, 0].plot(df['week'], df['vix'], 'tab:green')
axs[1, 0].set_title('VIX')
axs[1, 1].plot(df['week'], df['sp500'], 'tab:red')
axs[1, 1].set_title('S&P500')

## Train / Test Split

To make Bayesian priors comparable across regressors, we need some transformation across regressors. For simplicity, `MixMaxScalar()` is used here. In practice, one can consider standardizing process instead.

In [ ]:
test_size = 12
train_df = df[:-test_size].reset_index(drop=True)
test_df = df[-test_size:].reset_index(drop=True)

scaler = MinMaxScaler()
train_df[['claims', 'trend.unemploy', 'trend.job', 'sp500', 'vix']] = scaler.fit_transform(train_df[['claims', 'trend.unemploy', 'trend.job', 'sp500', 'vix']])
test_df[['claims', 'trend.unemploy', 'trend.job', 'sp500', 'vix']] = scaler.transform(test_df[['claims', 'trend.unemploy', 'trend.job', 'sp500', 'vix']])

# Simple DLT Model

In [ ]:
dlt = DLT(
    response_col=response_col,
    date_col=date_col,
    seasonality=52,
    num_warmup=4000,
    num_sample=1000,
    estimator='stan-mcmc',
    seed=2022,
)

dlt.fit(df=train_df)
predicted_df = dlt.predict(df=test_df)

In [ ]:
_ = plot_predicted_data(training_actual_df=train_df, 
                        predicted_df=predicted_df, 
                        date_col=date_col, 
                        actual_col=response_col, 
                        test_actual_df=test_df,
                        title='DLT with Linear Global Trend')

# DLT With Regression

In this session, a regression component is added on top of the `DLT` model.

The regressor columns can be supplied via argument `regressor_col`.  Recall the regression formula in **DLT**:

$$
\hat{y}_t =\mu_t + s_t + r_t \\
r_t = \sum_{j}\beta_j x_{jt} \\
\beta_j ~\sim \mathcal{N}(\mu_j, \sigma_j^2)
$$

Let's use the default (non-informative priors) where $\mu_j = 0$ and $\sigma_j = 1$.

## Regular Regression

In [ ]:
dlt_reg = DLT(
    response_col=response_col, 
    date_col=date_col,
    regressor_col=['trend.unemploy', 'trend.job', 'sp500', 'vix'],
    seasonality=52,
    num_warmup=4000,
    num_sample=1000,
    estimator='stan-mcmc',
    seed=2022,
)

dlt_reg.fit(df=train_df)
predicted_df_reg = dlt_reg.predict(test_df, decompose=True)

In [ ]:
_ = plot_predicted_data(training_actual_df=train_df, predicted_df=predicted_df_reg, 
                        date_col=date_col, actual_col=response_col, test_actual_df=test_df,
                        use_orbit_style=False,
                        title='DLT with Regular Regresion')

The estimated regressor coefficients can be retrieved via `.get_regression_coefs()`.

In [ ]:
dlt_reg.get_regression_coefs()

# Diagnoses

## Decomposition 

`plot_predicted_components` is the utility to plot each component separately. This is useful when one wants to look into the model prediction results and inspect each component separately.

In [ ]:
_ = plot_predicted_components(predicted_df_reg, date_col, 
                              use_orbit_style=False,
                              plot_components=['prediction', 'trend', 'seasonality', 'regression'])

## Posterior Diagnostic Visualizations

A few diagnostic plots is recommended for Orbit models. With `version>=1.1.0`,

These plots can be created directly in [Arivz](https://github.com/arviz-devs/arviz) which is a Python package for exploratory analysis of Bayesian models, includes functions for posterior analysis, data storage, model checking, comparison and diagnostics. Two plots can be found below:

1. Trace plot
2. Pair density plot

-

In [ ]:
import arviz as az
az.style.use('arviz-darkgrid')

### Trace plot

Trace plot shows the iterations of each paramter over the Markov chian sampling process. Trace plots provide an important tool for assessing mixing of a chain.

In [ ]:
ps = dlt_reg.get_posterior_samples(relabel=True, permute=False)
ps.keys()

In [ ]:
az.plot_trace(
    ps,
    var_names=['trend.unemploy', 'trend.job', 'sp500', 'vix', 'obs_sigma'],
    chain_prop={"color": ['r', 'b', 'g', 'y']},
    figsize=(10, 8),
);

### Pair density plot

We can also check the density of samples by pair plot.

In [ ]:
# az.plot_pair(
#     ps,
#     var_names=['trend.unemploy', 'trend.job', 'sp500', 'vix', 'obs_sigma'],
#     kind=["scatter", "kde"],
#     marginals=True,
#     point_estimate="median",
#     textsize=10.5,
#     figsize=(8,8)
# );

# Regression with Informative Priors / Regularized Priors

Due to various reasons, users may obtain further knowledge on some of the regressors or they want to propose different regularization on different regressors. These informative priors basically means to replace the defaults ($\mu$, $\sigma$) mentioned previously. In orbit, this process is done via the arguments `regressor_beta_prior` and `regressor_sigma_prior`. These two lists should be of the same length as `regressor_col`.

In addition, we can set a *sign* constraint for each coefficient $\beta_j$.  This is can be done by supplying the `regressor_sign` as a list where elements are in one of followings:

* '=': $\beta_j ~\sim \mathcal{N}(0, \sigma_j^2)$  i.e. $\beta_j \in (-\inf, \inf)$
* '+': $\beta_j ~\sim \mathcal{N}^+(0, \sigma_j^2)$  i.e. $\beta_j \in [0, \inf)$
* '-': $\beta_j ~\sim \mathcal{N}^-(0, \sigma_j^2)$  i.e. $\beta_j \in (-\inf, 0]$

Based on intuition, it's reasonable to assume search terms such as "unemployment", "filling" and **VIX** index to be positively correlated (`+` sign is used in this case) and upward shock of **SP500** (`-` sign) to be negatively correlated to the outcome. Otherwise, an unbounded coefficient can be used (`=` sign).

Furthermore, regressors such as seach queries may have more direct impact than stock marker indices.  Hence, a smaller $\sigma$ is considered.

In [ ]:
dlt_reg_adjust = DLT(
    response_col=response_col,
    date_col=date_col,
    regressor_col=['trend.unemploy', 'trend.job', 'sp500','vix'],
    regressor_sign=['+','=','-','+'],
    regressor_sigma_prior=[0.3, 0.1, 0.05, 0.1],
    num_warmup=4000,
    num_sample=1000,
    estimator='stan-mcmc',
    seed=2022,
)
dlt_reg_adjust.fit(df=train_df)
predicted_df_reg_adjust = dlt_reg_adjust.predict(test_df, decompose=True)

In [ ]:
_ = plot_predicted_data(training_actual_df=train_df, predicted_df=predicted_df_reg_adjust, 
                        date_col=date_col, actual_col=response_col, test_actual_df=test_df,
                        use_orbit_style=False,
                        title='DLT with Regresion of Informative Priors')

In [ ]:
dlt_reg_adjust.get_regression_coefs()

In [ ]:
naive_mae = mae(predicted_df['prediction'].values, test_df['claims'].values)
reg_mae = mae(predicted_df_reg['prediction'].values, test_df['claims'].values)
reg_adjust_mae = mae(predicted_df_reg_adjust['prediction'].values, test_df['claims'].values)

print("----------------Mean Absolute Error Summary----------------")
print("Naive Model: {:.3f}\nRegression Model: {:.3f}\nRefined Regression Model: {:.3f}".format(
    naive_mae, reg_mae, reg_adjust_mae
))

## Summary

This demo showcases a use case in nowcasting. Although this may not be applicable in real-time forecasting, it mainly introduces the regression analysis with time-series modeling in `Orbit`. For people who have concerns on the forecastability, one can consider introducing lag on regressors.

Also, `Orbit` allows informative priors where sometime can be useful in combining multiple source of insights together.